In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
save_dir = '/content/drive/MyDrive/quant'
os.makedirs(save_dir, exist_ok=True)  # 不存在则创建

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch torchvision onnx onnxruntime-gpu tensorrt
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 12.3 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1.2-cp312-cp312-linux_x86_64.whl size=659050 sha256=aa48dd2bcd85c430b132212117485cbf349a63b6d35606f23eea55a12e596889
  Stored in directory: /root/.cache/pip/wheels/d5/36/f3/ac5f09d768cad3fa15d5a3449bdfe65c3de58e69d036c73228
Successfully built pycuda


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
import tensorrt as trt
import onnx
from torch.utils.data import DataLoader, Subset
from onnx import shape_inference

# ==================== 配置 ====================
# 模型路径（需替换为你的模型路径）
MODEL_STATE_PATH = "/content/drive/MyDrive/quant/resnet18_final_model.pth"  # 训练好的PyTorch模型权重
ONNX_MODEL_PATH = "/content/drive/MyDrive/quant/resnet18_cifar100.onnx"     # 导出的ONNX模型
TRT_ENGINE_PATH = "/content/drive/MyDrive/quant/resnet18_int4_int8.engine"  # 生成的TensorRT引擎
CALIB_DATA_PATH = "/content/drive/MyDrive/quant/calib_data/"                # 校准数据保存路径（用于INT4量化）
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {DEVICE}")


# ==================== 模型定义（与训练时一致） ====================
class PretrainedResNet18(nn.Module):
    def __init__(self, num_classes=100):
        super().__init__()
        self.backbone = torchvision.models.resnet18(pretrained=False)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
        self.backbone.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.backbone.maxpool = nn.Identity()

    def forward(self, x):
        return self.backbone(x)


# ==================== 数据加载（用于校准和测试） ====================
def get_calibration_test_data():
    """获取校准数据（用于TensorRT量化）和测试数据"""
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    ])

    # 加载CIFAR-100测试集
    test_dataset = torchvision.datasets.CIFAR100(
        root='/content/drive/MyDrive/quant/data', train=False, download=True, transform=transform
    )

    # 划分校准集（1000张，用于INT4量化校准）和测试集
    calib_dataset = Subset(test_dataset, range(1000))
    test_dataset = Subset(test_dataset, range(1000, len(test_dataset)))

    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

    return calib_loader, test_loader, test_dataset.dataset.classes


# ==================== 步骤1：将PyTorch模型导出为ONNX ====================
def export_onnx_model():
    """将训练好的ResNet-18导出为ONNX格式"""
    # 初始化模型并加载权重
    model = PretrainedResNet18(num_classes=100).to(DEVICE)
    model.load_state_dict(torch.load(MODEL_STATE_PATH, map_location=DEVICE))
    model.eval()

    # 生成输入示例（CIFAR-100为32x32图像）
    dummy_input = torch.randn(1, 3, 32, 32).to(DEVICE)  # 批次大小=1，通道=3，高=32，宽=32

    # 导出ONNX
    torch.onnx.export(
        model,
        dummy_input,
        ONNX_MODEL_PATH,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},  # 支持动态批次
        opset_version=13,  # 推荐使用13+版本以支持更多算子
        do_constant_folding=True  # 折叠常量以优化模型
    )

    # 验证ONNX模型有效性
    onnx_model = onnx.load(ONNX_MODEL_PATH)
    onnx.checker.check_model(onnx_model)  # 检查模型完整性
    inferred_model = shape_inference.infer_shapes(onnx_model)  # 推理形状信息
    onnx.save(inferred_model, ONNX_MODEL_PATH)  # 保存优化后的模型

    print(f"ONNX模型导出成功: {ONNX_MODEL_PATH}")


# ==================== 步骤2：准备TensorRT校准数据（用于INT4量化） ====================
def prepare_calibration_data(calib_loader):
    """生成校准数据（TensorRT需要用真实数据计算量化范围）"""
    os.makedirs(CALIB_DATA_PATH, exist_ok=True)
    for i, (inputs, _) in enumerate(calib_loader):
        # 保存校准数据为numpy格式（TensorRT校准器需要）
        np.save(os.path.join(CALIB_DATA_PATH, f"calib_{i}.npy"), inputs.numpy())
        if i >= 31:  # 保存32个批次即可满足校准需求
            break
    print(f"校准数据已保存至: {CALIB_DATA_PATH}")


# ==================== 步骤3：实现TensorRT校准器（用于INT4量化） ====================
class Calibrator(trt.IInt8EntropyCalibrator2):
    def __init__(self, calib_data_path, batch_size=32):
        super().__init__()
        self.calib_data_path = calib_data_path
        self.batch_size = batch_size
        self.files = [f for f in os.listdir(calib_data_path) if f.endswith(".npy")]
        self.current_idx = 0

        # 分配设备内存存储校准数据
        self.device_input = torch.zeros((batch_size, 3, 32, 32), dtype=torch.float32, device=DEVICE).contiguous()
        self.input_ptr = self.device_input.data_ptr()

    def get_batch_size(self):
        return self.batch_size

    def get_batch(self, names):
        if self.current_idx >= len(self.files):
            return None  # 校准结束

        # 加载一批校准数据
        data = np.load(os.path.join(self.calib_data_path, self.files[self.current_idx]))
        self.current_idx += 1

        # 复制数据到设备内存
        self.device_input.copy_(torch.from_numpy(data).to(DEVICE))
        return [self.input_ptr]

    def read_calibration_cache(self):
        # 读取缓存的校准结果（加速后续量化）
        if os.path.exists("calib_cache.bin"):
            with open("calib_cache.bin", "rb") as f:
                return f.read()
        return None

    def write_calibration_cache(self, cache):
        # 保存校准结果到缓存
        with open("calib_cache.bin", "wb") as f:
            f.write(cache)


# ==================== 步骤4：使用TensorRT生成INT4+INT8混合精度引擎 ====================
def build_int4_int8_engine():
    """构建INT4+INT8混合精度的TensorRT引擎"""
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)  # 日志级别：WARNING（减少输出）
    builder = trt.Builder(TRT_LOGGER)
    network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
    parser = trt.OnnxParser(network, TRT_LOGGER)

    # 解析ONNX模型
    with open(ONNX_MODEL_PATH, "rb") as f:
        if not parser.parse(f.read()):
            print("ONNX模型解析失败：")
            for error in range(parser.num_errors):
                print(parser.get_error(error))
            return None

    # 配置构建参数
    config = builder.create_builder_config()
    config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)  # 1GB工作空间

    # 配置INT4+INT8混合精度量化
    # 1. 创建量化配置文件（指定哪些层用INT4，哪些用INT8）
    profile = builder.create_quantization_profile()
    input_tensor = network.get_input(0)

    # 2. 设置输入量化（INT8，通常输入不适合INT4）
    profile.set_input_quantization_scale(input_tensor.name, 1.0)  # 后续由校准器自动计算

    # 3. 启用INT4量化（TensorRT会自动决定哪些层用INT4/INT8以平衡精度和速度）
    config.add_quantization_profile(profile)
    config.set_flag(trt.BuilderFlag.INT8)  # 基础量化模式为INT8
    config.set_flag(trt.BuilderFlag.INT4)  # 启用INT4支持（混合精度）

    # 4. 设置校准器（用于计算量化范围）
    calibrator = Calibrator(CALIB_DATA_PATH)
    config.int8_calibrator = calibrator

    # 构建引擎
    serialized_engine = builder.build_serialized_network(network, config)
    if not serialized_engine:
        print("引擎构建失败")
        return None

    # 保存引擎
    with open(TRT_ENGINE_PATH, "wb") as f:
        f.write(serialized_engine)
    print(f"INT4+INT8混合精度引擎生成成功: {TRT_ENGINE_PATH}")
    return serialized_engine


# ==================== 步骤5：测试TensorRT量化模型 ====================
def test_trt_engine(engine):
    """测试量化后的TensorRT引擎性能"""
    # 准备测试数据
    _, test_loader, class_names = get_calibration_test_data()

    # 创建TensorRT运行时环境
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
    runtime = trt.Runtime(TRT_LOGGER)
    context = engine.create_execution_context()

    # 分配输入/输出内存（CPU和GPU）
    input_shape = (128, 3, 32, 32)  # 批次大小=128（需与测试集一致）
    output_shape = (128, 100)       # 输出维度=100（CIFAR-100类别数）

    # 绑定输入/输出索引
    input_idx = engine.get_binding_index("input")
    output_idx = engine.get_binding_index("output")

    # 分配CPU内存
    input_host = np.zeros(input_shape, dtype=np.float32)
    output_host = np.zeros(output_shape, dtype=np.float32)

    # 分配GPU内存
    input_device = cuda.mem_alloc(input_host.nbytes)
    output_device = cuda.mem_alloc(output_host.nbytes)
    stream = cuda.Stream()

    # 测试模型
    total = 0
    correct = 0
    for inputs, targets in test_loader:
        batch_size = inputs.shape[0]
        if batch_size < input_shape[0]:
            # 处理最后一个批次（可能不满128）
            input_host = np.zeros((batch_size, 3, 32, 32), dtype=np.float32)
            output_host = np.zeros((batch_size, 100), dtype=np.float32)

        # 复制输入数据到CPU内存
        input_host[:batch_size] = inputs.numpy()

        # 数据从CPU复制到GPU
        cuda.memcpy_htod_async(input_device, input_host, stream)

        # 设置动态批次大小
        context.set_binding_shape(input_idx, (batch_size, 3, 32, 32))

        # 执行推理
        bindings = [int(input_device), int(output_device)]
        context.execute_async_v2(bindings, stream.handle)

        # 结果从GPU复制到CPU
        cuda.memcpy_dtoh_async(output_host, output_device, stream)
        stream.synchronize()  # 等待GPU完成

        # 计算准确率
        preds = np.argmax(output_host[:batch_size], axis=1)
        correct += (preds == targets.numpy()).sum()
        total += batch_size

    accuracy = 100. * correct / total
    print(f"\nTensorRT INT4+INT8量化模型准确率: {accuracy:.2f}%")
    print(f"测试样本数: {total}")
    return accuracy


# ==================== 主流程 ====================
if __name__ == "__main__":
    # 检查CUDA和TensorRT是否可用
    if not torch.cuda.is_available():
        print("错误：需要NVIDIA GPU支持TensorRT量化")
        exit(1)

    try:
        import pycuda.driver as cuda
        import pycuda.autoinit  # 自动初始化CUDA
    except ImportError:
        print("请安装pycuda：pip install pycuda")
        exit(1)

    # 步骤1：导出ONNX模型
    export_onnx_model()

    # 步骤2：准备校准数据
    calib_loader, _, _ = get_calibration_test_data()
    prepare_calibration_data(calib_loader)

    # 步骤3：构建INT4+INT8引擎
    engine = build_int4_int8_engine()
    if not engine:
        exit(1)

    # 步骤4：测试量化模型
    test_trt_engine(engine)

    print("\nINT4+INT8混合精度量化流程完成！")

使用设备: cuda


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipython-input-2279765454.py:77: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.ht

ONNX模型导出成功: /content/drive/MyDrive/quant/resnet18_cifar100.onnx


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
